In [1]:
import skbio
import empress
import unittest
import pandas as pd

In [2]:
df = skbio.TreeNode.read('85_otus.tree')

In [3]:
from skbio import TreeNode

In [4]:
def treenode_to_dataframe(tree_node):
    rows_for_dataframe = []
    tree_node.assign_ids()
    for node in tree_node.traverse(include_self = True):
        '''
        if node.parent == None:
            parent_id = -1
        else:
            parent_id = node.parent

        node_id = node.id
        node_name = node.name
        node_length = node.length

        rows_for_dataframe.append({'parent':parent_id,'node':node_id,'name':node_name,'length':node_length})
        '''
        # if node.nazme == 'None':
            
        rows_for_dataframe.append({'parent':node.parent.id if node.parent is not None else -1,"node":node.id, 'name':node.name,'length':node.length})
        # add in name of the node and edge length of the node
    df = pd.DataFrame(rows_for_dataframe)
    return df

In [5]:
def dataframe_to_treenode(dataframe):
    #if 'node' not in dataframe.columns:
       # df[column_name] = default_value
    root_id = dataframe.loc[dataframe["parent"] == -1,"node"].values[0]
    root_name = dataframe.loc[dataframe["parent"] == -1,"name"].values[0]
    
    root = TreeNode(name = root_name) if root_name != 'None' else TreeNode()
    root.id = root_id
    dictionary_1 = {root.id:root}
    
    list_1 = dataframe['parent'].unique()
    list_2 = []
    
    for x in list_1:
        if x==-1:
            continue
        else:
            l2 = dataframe.loc[dataframe.parent == x,['parent','node','name','length']].values.tolist()
            for y in l2:
                if y[2] != 'None': # name does not equal None
                    node_name = y[2]
                    if pd.isna(y[3]): # if length does not equal Nan
                        node = TreeNode(name = node_name)
                        node.id = y[1]
                        dictionary_1[node.id] = node
                        #print(dictionary_1[y[0]])
                        dictionary_1[y[0]].append(node)
                    else:
                        node_length = float(y[3])
                        node = TreeNode(name = node_name,length = node_length)
                        node.id = y[1]
                        dictionary_1[node.id] = node
                        #print(dictionary_1[y[0]])
                        dictionary_1[y[0]].append(node)
                else: # if name = None
                    if pd.isna(y[3]): # if length = Nan
                        node = TreeNode()
                        node.id = y[1]
                        dictionary_1[node.id] = node
                        #print(dictionary_1[y[0]])
                        dictionary_1[y[0]].append(node)
                    else: # if length does not equal Nan
                        node_length = float(y[3])
                        node = TreeNode(length = node_length)
                        node.id = y[1]
                        dictionary_1[node.id] = node
                        #print(dictionary_1[y[0]])
                        dictionary_1[y[0]].append(node)

    return root

In [6]:
from unittest import TestCase
class unit_test_dataframe_to_treenode(TestCase):

    def setUp(self):
        node_a = TreeNode('a')
        node_b = TreeNode('b')
        node_c = TreeNode('c')
        node_d = TreeNode('d')
        node_e = TreeNode('e')
        node_a.append(node_b)
        node_b.append(node_c)
        node_b.append(node_d)
        node_a.append(node_e)
        self.TreeRoot = node_a
    #
    #          /c
    #          |
    #      /b--|
    #      |   |
    #--a---|   \d
    #      |
    #      \e

        list_df = [{'parent':-1,'node':1,'name':'a','length':None},
                   {'parent':1,'node':2,'name':'b','length':None},
                   {'parent':2,'node':3,'name':'c','length':None},
                   {'parent':2,'node':4,'name':'d','length':None},
                   {'parent':1,'node':5,'name':'e','length':None}
                  ]
        data_frame = pd.DataFrame(data=list_df)
        self.test_tree = dataframe_to_treenode(data_frame)
        #self.expected_tree = skbio.TreeNode.read('85_otus.tree', format='newick')
        #observed_tree = treenode_to_dataframe(self.expected_tree)
        #self.observed_tree = dataframe_to_treenode(observed_tree)

    def test_count_tips(self):
        real_tree_tips = 3
        test_tree_tips = self.test_tree.count(tips = True)
        self.assertEqual(test_tree_tips,real_tree_tips)

    def test_root(self):
        real_root = 'a'
        test_tree_root = self.test_tree.root().name
        self.assertEqual(test_tree_root,real_root)

    def test_tip(self):
        real_tips = {'c','d','e'}
        #test_tree_tips = list(self.test_tree.tips())
        est_tree_tips = {n.name for n in self.test_tree.tips()}
        self.assertEqual(real_tips,est_tree_tips)
        
    
        #print(self.test_tree.ascii_art())

    '''
    def test_rfd(self):
        #self.expected_tree.assign_ids()
        #print(type(self.expected_tree))
        self.assertEqual(self.expected_tree.compare_rfd(self.observed_tree), 0.0)
        #print(self.expected_tree)
        
        
    def test_tip_distance(self):
        self.expected_tree.assign_ids()
        self.assertEqual(self.expected_tree.compare_tip_distances(self.observed_tree,sample = 1000), 0.0) 
    
    def test_paths(self):
        expeted_tips = {expeted_tips for expeted_tips in self.expected_tree.tips()}
        observed_tips = {observed_tips for observed_tips in self.observed_tree.tips()}
        if expeted_tips != observed_tips:
            self.assertEqual(0,1)
        for exp_tips in expeted_tips:
            e_tips = self.expected_tree.find(exp_tips)
            o_tips = self.observed_tree.find(exp_tips)
            ancestor_exp = [node.name for node in e_tips.ancestors()]
            ancestor_obs = [node.name for node in o_tips.ancestors()]
            if ancestor_exp != ancestor_obs:
                self.assertEqual(0,1)
        self.assertEqual(1,1)
    '''
#unittest.main()
unittest.main(argv=[''], verbosity=2, exit=False)

test_count_tips (__main__.unit_test_dataframe_to_treenode.test_count_tips) ... ok
test_root (__main__.unit_test_dataframe_to_treenode.test_root) ... ok
test_tip (__main__.unit_test_dataframe_to_treenode.test_tip) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.250s

OK


In [16]:
class unit_test_dataframe_to_treenode(TestCase):

    def setUp(self):
        node_a = TreeNode('a')
        node_b = TreeNode('b')
        node_c = TreeNode('c')
        node_d = TreeNode('d')
        node_e = TreeNode('e')
        node_a.append(node_b)
        node_b.append(node_c)
        node_b.append(node_d)
        node_a.append(node_e)
        self.TreeRoot = node_a
        #
        #          /c
        #          |
        #      /b--|
        #      |   |
        # --a---|   \d
        #      |
        #      \e

        list_df = [{'parent': -1, 'node': 1, 'name': 'a', 'length': None},
                   {'parent': 1, 'node': 2, 'name': 'b', 'length': None},
                   {'parent': 2, 'node': 3, 'name': 'c', 'length': None},
                   {'parent': 2, 'node': 4, 'name': 'd', 'length': None},
                   {'parent': 1, 'node': 5, 'name': 'e', 'length': None}
                   ]

        self.expected_tree = skbio.TreeNode.read('85_otus.tree', format='newick')
        observed_tree = treenode_to_dataframe(self.expected_tree)
        self.observed_tree = dataframe_to_treenode(observed_tree)
    
    def test_count_tips(self):
        real_tree_tips = self.expected_tree.count(tips=True)
        test_tree_tips = self.observed_tree.count(tips=True)
        self.assertEqual(test_tree_tips, real_tree_tips)

    def test_root(self):
        real_root = self.expected_tree.root().name
        test_tree_root = self.observed_tree.root().name
        self.assertEqual(test_tree_root, real_root)

    def test_tip(self):
        real_tips = {n.name for n in self.expected_tree.tips()}
        # test_tree_tips = list(self.test_tree.tips())
        est_tree_tips = {n.name for n in self.observed_tree.tips()}
        self.assertEqual(real_tips, est_tree_tips)

        # print(self.test_tree.ascii_art())


    def test_rfd(self):
        #self.expected_tree.assign_ids()
        #print(type(self.expected_tree))
        self.assertEqual(self.expected_tree.compare_rfd(self.observed_tree), 0.0)
        #print(self.expected_tree)


    def test_tip_distance(self):
        self.expected_tree.assign_ids()
        self.assertEqual(self.expected_tree.compare_tip_distances(self.observed_tree,sample = 1000), 0.0) 
    
    def test_paths(self):
        observed_tips = {tip.name for tip in self.observed_tree.tips()}
        #expeted_tips = {expeted_tips for expeted_tips in self.expected_tree.tips()}
        #observed_tips = {observed_tips for observed_tips in self.observed_tree.tips()}
        expected_tips = {tip.name for tip in self.expected_tree.tips()}
        if expected_tips != observed_tips:
            self.assertEqual(0,1)
        for exp_tips in expected_tips:
            e_tips = self.expected_tree.find(exp_tips)
            o_tips = self.observed_tree.find(exp_tips)
            ancestor_exp = [node.name for node in e_tips.ancestors()]
            ancestor_obs = [node.name for node in o_tips.ancestors()]
            if ancestor_exp != ancestor_obs:
                self.assertEqual(0,1)
        self.assertEqual(1,1)
    


# unittest.main()
unittest.main(argv=[''], verbosity=2, exit=False)

test_count_tips (__main__.unit_test_dataframe_to_treenode.test_count_tips) ... ok
test_paths (__main__.unit_test_dataframe_to_treenode.test_paths) ... ok
test_rfd (__main__.unit_test_dataframe_to_treenode.test_rfd) ... ok
test_root (__main__.unit_test_dataframe_to_treenode.test_root) ... ok
test_tip (__main__.unit_test_dataframe_to_treenode.test_tip) ... ok
test_tip_distance (__main__.unit_test_dataframe_to_treenode.test_tip_distance) ... /Users/shebinsam/Desktop/knight_lab/env/lib/python3.12/site-packages/skbio/tree/_tree.py:3287: RepresentationWarning: `TreeNode.tip_tip_distances`: Node with name 'k__Bacteria' does not have an associated length, so a length of 0.0 will be used.
  warn(
ok

----------------------------------------------------------------------
Ran 6 tests in 20.600s

OK


In [13]:
class unit_test_treenode_to_dataframe(TestCase):
    
    def setUp(self):
        node_a = TreeNode('a')
        self.test_tree = node_a
        node_b = TreeNode('b')
        node_c = TreeNode('c')
        node_d = TreeNode('d')
        node_e = TreeNode('e')
        node_a.append(node_b)
        node_b.append(node_c)
        node_b.append(node_d)
        node_a.append(node_e)
        self.TreeRoot = node_a
    #
    #          /c
    #          |
    #      /b--|
    #      |   |
    #--a---|   \d
    #      |
    #      \e

        node_a.assign_ids()
        self.data_frame = treenode_to_dataframe(node_a)
        
        self.parent_list = []
        self.node_list = []
        for node in node_a.traverse(include_self = True):
            self.parent_list.append(node.parent.id if node.parent is not None else -1)
            self.node_list.append(node.id)
        
            

    def test_parent(self):
        test_parent_list = [x for x in self.data_frame['parent']]
        self.assertEqual(test_parent_list,self.parent_list)

    def test_node(self):
        test_node_list = [y for y in self.data_frame['node']]
        self.assertEqual(self.node_list,test_node_list)
    
    def test_parent_size(self):
        parent_len = len(self.data_frame['parent'])
        self.assertEqual(parent_len,5)
        
    def test_node_size(self):
        node_len = len(self.data_frame['node'])
        self.assertEqual(node_len,5)
    
                          
unittest.main(argv=[''], verbosity=2, exit=False)

test_count_tips (__main__.unit_test_dataframe_to_treenode.test_count_tips) ... ok
test_root (__main__.unit_test_dataframe_to_treenode.test_root) ... ok
test_tip (__main__.unit_test_dataframe_to_treenode.test_tip) ... ok
test_node (__main__.unit_test_treenode_to_dataframe.test_node) ... ok
test_node_size (__main__.unit_test_treenode_to_dataframe.test_node_size) ... ok
test_parent (__main__.unit_test_treenode_to_dataframe.test_parent) ... ok
test_parent_size (__main__.unit_test_treenode_to_dataframe.test_parent_size) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.107s

OK


## Test for TreeNode to DataFrame

In [69]:
node_a = TreeNode()
node_b = TreeNode(name ='b', length = 2)
node_c = TreeNode(name = 'c', length = 1)
node_d = TreeNode('d')
node_e = TreeNode(length = 1)
node_a.append(node_b)
node_b.append(node_c)
node_b.append(node_d)
node_a.append(node_e)
data_frame = treenode_to_dataframe(node_a)

In [70]:
print(node_a.ascii_art())

                    /-c
          /b-------|
---------|          \-d
         |
          \-


In [71]:
data_frame

,parent,node,name,length
0,-1,4,None,NaN
1,4,2,b,2.0
2,2,0,c,1.0
3,2,1,d,NaN
4,4,3,None,1.0


In [14]:
type(data_frame['length'][0])

numpy.float64

In [15]:
data_frame['length'][0] == float('NaN')

False

In [16]:
pd.isna(data_frame['length'][0])

True

In [17]:
type(data_frame['name'][0])

NoneType

In [18]:
data_frame['name'][0] == None

True

In [81]:
pd.isna(data_frame['name'][0])

True

In [75]:
a = dataframe_to_treenode(data_frame)

2.0
1.0
1.0
nan


In [76]:
print(a.ascii_art())

                    /-c
          /b-------|
---------|          \-d
         |
          \-


In [77]:
print(a.count(tips= True))

3
